In [1]:
def decorated_by(func):
    func.__doc__ += '\n it decorated by decorated_by'
    return func

@decorated_by
def add(x, y):
    '''return the sum of x and y'''
    return x + y

help(add)

Help on function add in module __main__:

add(x, y)
    return the sum of x and y
    it decorated by decorated_by



In [1]:
def also_decorated_by(func):
    func.__doc__ += '\n it decorated by also_decorated_by'
    return func

def decorated_by(func):
    func.__doc__ += '\n it decorated by decorated_by'
    return func

@also_decorated_by
@decorated_by
def add(x, y):
    '''return the sum of x and y'''
    return x + y

help(add)

Help on function add in module __main__:

add(x, y)
    return the sum of x and y
    it decorated by decorated_by
    it decorated by also_decorated_by



# A func register

In [3]:
registry = []

def register(func):
    registry.append(func)
    return func

@register
def foo1():
    return 5
@register
def foo2():
    return 3

answer = 0
for func in registry:
    answer += func()

answer

8

# Separate registry

In [9]:
class Registry:
    def __init__(self):
        self._functions = []
    def register(self, func):
        print(func)
        self._functions.append(func)
        return func
    def run_all(self, *args, **kwargs):
        return_value = []
        for func in self._functions:
            return_value.append(func(*args, **kwargs))
        return return_value
    
a = Registry()
b = Registry()

@a.register
def foo(x = 3):
    return x

@a.register
@b.register
def bar(x=5):
    return x

print(a.run_all(x = 6))
print(b.run_all(x=4))
print(a.run_all())

<function foo at 0x00000192B777B5E0>
<function bar at 0x00000192B7770820>
<function bar at 0x00000192B7770820>
[6, 6]
[4]
[3, 5]


# a simple type check

In [13]:
def requires_ints(func):
    def inner(*args, **kwargs):
        kwarg_values = [i for i in kwargs.values()]
        
        for arg in list(args) + kwarg_values:
            if not isinstance(arg, int):
                raise TypeError('we need int')
        return func(*args, **kwargs)
    return inner

@requires_ints
def foo(x, y, *args):
    print(x + y)
    
foo(1,2,3,4)
foo('1', '2')

3


TypeError: we need int

In [11]:
help(foo)

Help on function inner in module __main__:

inner(*args, **kwargs)



# Preserving the help

In [14]:
from functools import wraps

def requires_ints(func):
    @wraps(func)
    def inner(*args, **kwargs):
        kwarg_values = [i for i in kwargs.values()]
        
        for arg in list(args) + kwarg_values:
            if not isinstance(arg, int):
                raise TypeError('we need int')
        return func(*args, **kwargs)
    return inner

@requires_ints
def foo(x, y, *args):
    print(x + y)
    
help(foo)

Help on function foo in module __main__:

foo(x, y, *args)



# User Verification

In [3]:
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email

class AnonymousUser:
    def __init__(self):
        self.name = None
        self.email = None
    def __nonzero__(self):
        return False
    
import functools

def require_user(func):
    @functools.wraps(func)
    def inner(user, *args, **kwargs):
        if user and isinstance(user, User):
            return func(user, *args, **kwargs)
        else:
            raise ValueError("we need a user")
    return inner

@require_user
def foo(user, x, y):
    print(x + y)
    
foo(User('lzy', '1@qq.com'), 3, 5)
foo(AnonymousUser(), 3, 5)

8


ValueError: we need a user

# output formatting

In [11]:
import functools
import json
def json_output(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        return json.dumps(func(*args, **kwargs))
    return inner

@json_output
def foo():
    return {'a':1, 'b':2}

type(foo())

str

In [14]:
class JsonOutputError(Exception):
    def __init__(self, message):
        self._message = message
    def __str__(self):
        return self._message
    
def json_output(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        try:
            result = func(*args, **kwargs)
        except JsonOutputError as e:
            result = {'status': 'error',
                      'message': str(e)}
        return json.dumps(result)
    return inner

@json_output
def foo():
    raise JsonOutputError('json error')
#     return {'a':1, 'b':2}

foo()

'{"status": "error", "message": "json error"}'

# logging


In [16]:
import functools
import time
import logging

def logged(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        start = time.time()
        return_value = func(*args, **kwargs)
        delta = time.time() - start
        logger = logging.warning(f'{func.__name__} excute time {delta} return {return_value}')
        return return_value
    return inner

@logged
def foo(x):
    return x

foo(42)

42

# Decorator Args

In [24]:
import functools, json
class JsonOutputError(Exception):
    def __init__(self, message):
        self._message = message
    def __str__(self):
        return self._message

def json_output(indent=None, sort_keys=False):
    def actual_decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            try:
                result = func(*args, **kwargs)
            except JsonOutputError as e:
                result = {'status': 'error',
                          'message': str(e)}
            return json.dumps(result, indent=indent, sort_keys=sort_keys)
        return inner
    return actual_decorator

@json_output(indent=4)
def foo():
    raise JsonOutputError('json error')
#     return {'a':1, 'b':2}

print(foo())

{
    "status": "error",
    "message": "json error"
}


# The Call Signature Matters

In [26]:
import functools, json
class JsonOutputError(Exception):
    def __init__(self, message):
        self._message = message
    def __str__(self):
        return self._message

def json_output(decorated_=None, indent=None, sort_keys=False):
    if decorated_ and (indent or sort_keys):
        raise ValueError('unexpected args')
    def actual_decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            try:
                result = func(*args, **kwargs)
            except JsonOutputError as e:
                result = {'status': 'error',
                          'message': str(e)}
            return json.dumps(result, indent=indent, sort_keys=sort_keys)
        return inner
    
    if decorated_:
        return actual_decorator(decorated_)
    else:
        return actual_decorator

@json_output()
def foo():
    raise JsonOutputError('json error')
#     return {'a':1, 'b':2}

print(foo())

{"status": "error", "message": "json error"}


# test

In [45]:
import logging
import time
import functools
def logged(func=None, times=None):
    if func and times:
        raise ValueError('unexpected args')
    times = 1 if not times else times
    @functools.wraps(func)
    def actual_func(func):
        def inner(*args, **kwargs):
            for i in range(times):
                logging.warning(time.ctime())
            func(*args, **kwargs)
        return inner
    
    if func:
        return actual_func(func)
    else:
        return actual_func
    
@logged
def foo():
    a = 1
    
foo()

# Decorating Classes

In [61]:
import functools
import time

def sorted_by_creation_time(cls):
    origin_init = cls.__init__
    
    @functools.wraps(origin_init)
    def new_init(self, *args, **kwargs):
        origin_init(self, *args, **kwargs)
        self._create = time.time()
    cls.__init__ = new_init
    cls.__lt__ = lambda self, other : self._create < other._create
    return cls

@sorted_by_creation_time
class SortAble:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return self.name
    __str__ = __repr__

first = SortAble('first')
time.sleep(0.1)
second = SortAble('second')
print(first > second)

False


str